# Самокат. Tech

Вам предстоит решить часть задачи матчинга, связанную с отбором объектов. Матчинг — это процесс, который сопоставляет товары друг с другом. В самой задаче нужно будет грамотно обработать данные и оптимизировать векторный поиск для достижения максимального значения полноты.

## Dataset Description

В этом задании на вход будет подаваться векторное представление товаров из base и query. Задача - для каждого товара из query предложить 10 кандидатов на матч.
Данные:

base.csv - векторное представление товаров base
train.csv - векторное представление товаров query + таргет (товар из base, являющийся матчем)
test.csv - векторное представление товаров query, для которых надо предсказать кандидатов на матч из base
answer_sample.csv - формат ответа: Id - id продукта, Prtedicted - 10 id продуктов из base через пробел
baseline.ipynb - ноутбук с простым решением. Для адекватного времени выполнения в ноутбуке нужно кое-что поменять.

### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import re
import faiss
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)

In [11]:
import nbconvert

### Чтение данных

In [2]:
df_base = pd.read_csv('base.csv')

In [3]:
df_train = pd.read_csv('train.csv')

In [4]:
df_test = pd.read_csv('test.csv')

In [5]:
answer_sample = pd.read_csv('answer_sample.csv')

### Анализ корреляции признаков base и train 

In [32]:
# Объединение таблиц по ключевому признаку "Id"
merged_df = pd.merge(df_base, df_train, left_on="Id", right_on="Target", suffixes=("_base", "_train"))
merged_df = merged_df.drop(['Id_train', 'Target'], axis = 1)

# Масштабирование признаков от -1 до 1
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_df = pd.DataFrame(scaler.fit_transform(merged_df.iloc[:, 1:]))

# Функция для расчета коэффициента корреляции Пирсона и проверки условия
def calculate_correlation_pearson(base_column, train_column):
    correlation, _ = pearsonr(base_column, train_column)
    return correlation

# Функция для расчета коэффициента корреляции Спирмена и проверки условия
def calculate_correlation_spearman(base_column, train_column):
    correlation, _ = spearmanr(base_column, train_column)
    return correlation


# Списки для хранения названий столбцов с низким коэффициентом корреляции
low_correlation_columns = []
correlation_values_pearson = []
correlation_values_spearman = []

# Задайте пороговое значение для корреляции
correlation_threshold = 0.6

# Проход по столбцам параметров
for col_index in range(72):
    base_column = scaled_df.iloc[:, col_index]
    train_column = scaled_df.iloc[:, col_index + 72]
    
    correlation_pearson = calculate_correlation_pearson(base_column, train_column)
    correlation_spearman = calculate_correlation_spearman(base_column, train_column)
    #print (col_index,correlation_pearson, correlation_spearman)
    correlation_values_pearson.append(correlation_pearson)
    correlation_values_spearman.append(correlation_spearman)
    
    # Проверка условия и добавление названия столбца в список
    if correlation_pearson < correlation_threshold:
        low_correlation_columns.append(merged_df.columns[col_index + 1])

# Вывод списка столбцов с низким коэффициентом корреляции
print(low_correlation_columns)

'''
# Проход по столбцам параметров
for col_index in range(1, 73):
    base_column = merged_df.iloc[:, col_index]
    train_column = merged_df.iloc[:, col_index + 72]
    
    correlation_pearson = calculate_correlation_pearson(base_column, train_column)
    correlation_spearman = calculate_correlation_spearman(base_column, train_column)
    #print (col_index,correlation_pearson, correlation_spearman)
    correlation_values_pearson.append(correlation_pearson)
    correlation_values_spearman.append(correlation_spearman)
    
    # Проверка условия и добавление названия столбца в список
    if correlation_pearson < correlation_threshold:
        low_correlation_columns.append(merged_df.columns[col_index])
'''
# Расчет среднего значения и дисперсии корреляции
mean_correlation_pearson = sum(correlation_values_pearson) / len(correlation_values_pearson)
variance_correlation_pearson = sum((correlation_pearson - mean_correlation_pearson) ** 2 for correlation_pearson
                           in correlation_values_pearson) / len(correlation_values_pearson)

low_correlation_column_numbers = [re.findall(r'\d+', value)[0] for value in low_correlation_columns]
#print(low_correlation_column_numbers)

# Создание новых массивов без столбцов с низкой корреляцией
df_base_filtered = df_base.drop(low_correlation_column_numbers, axis = 1)
df_train_filtered = df_train.drop(low_correlation_column_numbers, axis = 1)
df_test_filtered = df_test.drop(low_correlation_column_numbers, axis = 1)

# Вывод списка столбцов с низким коэффициентом корреляции
print(low_correlation_columns)


['21_base', '25_base', '33_base', '59_base', '65_base']
['21_base', '25_base', '33_base', '59_base', '65_base']


### Поиск ближайших соседей с помощью FAISS

In [34]:
def match_embeddings(base, train):#, nlist=10000, nprobe=1000):
    base_embeddings = base.iloc[:, 1:].values#.astype('float32')
    train_embeddings = train.iloc[:, 1:].values#.astype('float32')
    #train_embeddings = train.iloc[:, 1:-1].values.astype('float32')  # Исключаем последний столбец Target, если используем Train
    
    # Создание индекса
    index = faiss.IndexFlatL2(base_embeddings.shape[1])  # Эвклидово расстояние
    #index = faiss.IndexFlatIP(base_embeddings.shape[1])   # Косинусное расстояние
    index.add(base_embeddings)
    
    '''
    # Параметры индекса
    nlist = 100  # Количество клеток в индексе
    quantizer = faiss.IndexFlatL2(base_embeddings.shape[1])
    index = faiss.IndexIVFFlat(quantizer, base_embeddings.shape[1], nlist, faiss.METRIC_INNER_PRODUCT)

    # Обучение индекса
    index.train(base_embeddings)
    index.add(base_embeddings)
    '''

    # Поиск ближайших соседей для эмбеддингов из train
    _, matched_indices = index.search(train_embeddings, k=10)

    matched_ids = train['Id'].values
    base_ids = base['Id'].values[matched_indices]

    return matched_ids, base_ids

In [35]:

matched_ids, base_ids = match_embeddings(df_base_filtered, df_test_filtered)
#matched_ids, base_ids = match_embeddings(df_base_filtered, df_test_filtered, nlist=10000, nprobe=10000)

# matched_ids, base_ids = match_embeddings(base_sampled, df_train) # Если испольуем Train, меняем train_embeddings в фунции match_embeddings

result = pd.DataFrame({'Id': [f'{qid.split("-")[0]:0>5}-query' for qid in matched_ids],
                       'Predicted': [' '.join(ids) for ids in base_ids]})


### Вывод результатов поиска

In [36]:
result

,Id,Predicted
0,100000-query,3839597-base 3857586-base 645855-base 3181043-...
1,100001-query,174378-base 29560-base 1016346-base 1045656-ba...
2,100002-query,472256-base 153272-base 979799-base 395020-bas...
3,100003-query,2104072-base 2968459-base 3221757-base 346795-...
4,100004-query,75484-base 682511-base 3520568-base 976469-bas...
...,...,...
99995,199995-query,4090339-base 1749950-base 1295135-base 2958365...
99996,199996-query,2653840-base 1555564-base 3672615-base 2020012...
99997,199997-query,2645169-base 2844734-base 75694-base 1200138-b...
99998,199998-query,341779-base 3485405-base 1341156-base 4572701-...


In [37]:
result.to_csv('answer.csv', index=False)

### Выводы:

В ходе работы был рассмотрена и применена библиотека FAISS для поиска ближайших соседей среди эмбеддингов.
Итоговый результат на закрытой тестовой выборке составил: *0.55586*. При этом опубликованное значение baseline модели на этой же выборке было: *0.13850*. Лидеру в данном соревновании удалось набрать *0.77120*

Таким образом, можно считать, что полученный результат значительно отличается от baseline, однако всё равно достаточно далёк от от результатов лидеров.

Дальнейшими путями для улучшения результата могут служить создание и обучение дополнительных моделей машинного обучения для ранжирования полученных результатов с целью выявления зависимостей в очерёдности выдачи. Получение близких запросу значений путём вычисления косинусных расстояний будет трудно выполнимо на практике ввиду большого количества элементов в датасете, а так же из-за большого количества признаков (72 столбца). 